# Тестирование и python

В этой лекции мы познакомимся тестированием на `python`, реализовав небольшую кату по TDD (test-driven developement).

## TDD и каты

TDD — test-driven developement или разработка через тестирование.

**Три правила TDD**:

 - Продакшн-код можно писать только для починки падающего теста.
 - В тесте нужно писать ровно столько кода, сколько необходимо, чтобы он упал. Ошибки компиляции считаются падениями теста.
 - В продакшн можно написать ровно столько кода, сколько требуется для починки одного падающего теста.


Получается следйющий пайплайн: пишем падающий тест, пишем код, чтобы тест не падал, рефакторим код так, чтобы тесты не падали. Повторяем до сходимости.

Есть пара книжек по теме:

1. [Test Driven Development: By Example 1st Edition](https://www.eecs.yorku.ca/course_archive/2003-04/W/3311/sectionM/case_studies/money/KentBeck_TDD_byexample.pdf)
2. [On Growing Object Oriented Software, Guided by Tests](https://www.amazon.com/Growing-Object-Oriented-Software-Guided-Tests/dp/0321503627)

К прочтению рекомендуется вторая, т.к. она более приближена к разработческим реалиям.

### Каты

Каты — упражнения по программированию, помогающие отточить навыки путем многократного повторнения. Концепция взята из японских боевых искусств. Подробнее про них можно почитать в книжке [The Pragmatic Programmer](https://pragprog.com/titles/tpp20/the-pragmatic-programmer-20th-anniversary-edition/)



**Ката Greeter**

Эту кату надо выполнять строго по пунктам, не заглядывая вперёд.

- Создайте класс `Greeter`, у которого есть метод `greet` принимающий на вход имя и возвращающий "Hello <имя>".

## 1. Первый тест

Для автоматизированного тестирования написано много фреймворков на разных языках. Короткий список для python:

* [unittest](https://docs.python.org/3/library/unittest.html)
* [nose2](https://docs.nose2.io/en/latest/)
* [pytest](https://docs.pytest.org/en/latest/)

В рамках лекции мы остановимся на `pytest`.

In [8]:
# Настраиваем ноутбук
import pytest
import ipytest
ipytest.autoconfig()

ModuleNotFoundError: No module named 'ipytest'

#### Как pytest находит тесты

1. Рекурсивно находит все python-файлы в текущей директории
2. Оставляет только файлы вида `test_*.py` и `*_test.py`
3. В этих файлах:
    1. Находит все функции с префиксом `test`
    2. Находит все методы с префиксом `test` внутри классов с префиксом `Test`. У классов не должно быть метода `__init__`
  
Поведение можно модифицировать. [Подробнее в документации](https://docs.pytest.org/en/stable/goodpractices.html#test-discovery).

Напишем минимальный тест:

In [6]:
%%ipytest

def test_greeter():
    Greeter()

UsageError: Cell magic `%%ipytest` not found.


`pytest` выводит отчет, в котором можно посмотреть сколько у нас всего тестов, какие из них упали и по какой причине. 

Теперь сделаем так, чтобы тест проходил:

In [ ]:
class Greeter:
    pass

In [ ]:
%%ipytest

def test_greeter():
    Greeter()

Еще одна итерация TDD:

In [ ]:
%%ipytest

def test_greeter():
    Greeter().greet("Mike")

In [ ]:
class Greeter:
    def greet(self, name):
        return ""

In [ ]:
%%ipytest

def test_greeter():
    Greeter().greet("Mike")

Теперь наконец-то напишем нормальный тест, воспользовавшись основной фишкой `Pytest`: `assert`. `Pytest` находит все вызовы `assert` в коде тестов, а затем переписывает этот код так, чтобы в случае падения пользователь мог получить удобный дифф и трейсбек.

[Демки разных ассертов](https://docs.pytest.org/en/stable/example/reportingdemo.html#tbreportdemo).

In [ ]:
%%ipytest

def test_greeter():
    assert Greeter().greet("Mike") == "Hello Mike"

Починим тест:

In [ ]:
class Greeter:
    def greet(self, name):
        return "Hello Mike"

In [ ]:
%%ipytest

def test_greeter():
    assert Greeter().greet("Mike") == "Hello Mike"

## 2. Параметризация

Наша реализация представляет собой немного не то, что мы хотели. Стоит добавить больше разных тестов.
Чтобы не копировать один и тот же тест, можно воспользоваться параметризацией:

In [ ]:
%%ipytest
test_cases = [("Mike", "Hello Mike"), ("John", "Hello John"), ("Greg", "Hello Greg")]

@pytest.mark.parametrize("name, greeting", test_cases)
def test_greeter(name, greeting):
    assert Greeter().greet(name) == greeting

Видим отчеты пайтеста во всей красе. Починим тесты:

In [ ]:
class Greeter:
    def greet(self, name):
        return "Hello " + name

In [ ]:
%%ipytest
test_cases = [("Mike", "Hello Mike"), ("John", "Hello John"), ("Greg", "Hello Greg")]

@pytest.mark.parametrize("name, greeting", test_cases)
def test_greeter(name, greeting):
    assert Greeter().greet(name) == greeting

Перейдем к следующему пункту нашего задания:

- Метод `greet` должен убирать лишние пробелы в начале и в конце имени.

Опять же, напишем тест:

In [ ]:
%%ipytest

def test_spaces():
    greeting = Greeter().greet(" Mike")
    assert not greeting.startswith(" ")

Обратим внимание, что тест проходит и возникает соблазн продолжить работу. Однако если посмотреть на тест внимательно, можно увидеть в нем ошибку.


Чтобы не наступать на такие грабли, существует **правило** — только что написанный тест должен падать, при чём именно из-за того поведения, которое этот тест должен был покрыть.


Вы можете писать тесты на уже существующий код — в таком случае они могут не падать, т.к. код уже работает как надо. Тогда есть два варианта:
* Сделать в продовом коде баг чтобы тест упал
* Обратить проверяемое условие в тесте

Поправим наш тест:

In [ ]:
%%ipytest

def test_spaces():
    greeted_name = Greeter().greet(" Mike").split(" ", 1)[1]
    assert not greeted_name.startswith(" ")

Починим тест

In [ ]:
class Greeter:
    def greet(self, name):
        if name.startswith(" "):
            name = name[1:] 
        return "Hello " + name

In [ ]:
%%ipytest

def test_spaces():
    greeted_name = Greeter().greet(" Mike").split(" ", 1)[1]
    assert not greeted_name.startswith(" ")

Перечитаем наше задание:
* Метод greet должен убирать лишние пробелы в начале и **в конце имени**

Видимо, нам нужно расширить тест:

In [ ]:
%%ipytest

def test_spaces():
    greeted_name = Greeter().greet(" Mike ").split(" ", 1)[1]
    assert not greeted_name.startswith(" ") and not greeted_name.endswith(" ")

Починим тест:

In [ ]:
class Greeter:
    def greet(self, name):
        if name.startswith(" "):
            name = name[1:]
        if name.endswith(" "):
            name = name[:-1]
        return "Hello " + name

In [ ]:
%%ipytest

def test_spaces():
    greeted_name = Greeter().greet(" Mike ").split(" ", 1)[1]
    assert not greeted_name.startswith(" ") and not greeted_name.endswith(" ")

Наш тест все еще недостаточно хорош. Хороший набор тестов должен покрывать разные граничные условия и заходить во все ветки исполнения кода. Параметризуем наш тест так, чтобы покрыть как можно ветвей исполнения кода:

In [ ]:
%%ipytest

@pytest.mark.parametrize("name", ["Mike", " Mike", "Mike ", " Mike ", "  Mike", "  Mike  "])
def test_spaces(name):
    greeted_name = Greeter().greet(name).split(" ", 1)[1]
    assert not greeted_name.startswith(" ") and not greeted_name.endswith(" ")

Можно давать имена отдельным наборам параметров — тогда будет удобнее читать вывод пайтеста

In [ ]:
%%ipytest

@pytest.mark.parametrize("name", ["Mike", " Mike", "Mike ", " Mike ", "  Mike", "  Mike  "],
                         ids=["no spaces", "left space", "right space", 
                              "two-side space", "double space", "two-sided double space"])
def test_spaces(name):
    greeted_name = Greeter().greet(name).split(" ", 1)[1]
    assert not greeted_name.startswith(" ") and not greeted_name.endswith(" ")

Починим тест:

In [ ]:
class Greeter:
    def greet(self, name):
        while name.startswith(" "):
            name = name[1:]
        while name.endswith(" "):
            name = name[:-1]
        return "Hello " + name

In [ ]:
%%ipytest

@pytest.mark.parametrize("name", ["Mike", " Mike", "Mike ", " Mike ", "  Mike", "  Mike  "],
                         ids=["no spaces", "left space", "right space", 
                              "two-side space", "double space", "two-sided double space"])
def test_spaces(name):
    greeted_name = Greeter().greet(name).split(" ", 1)[1]
    assert not greeted_name.startswith(" ") and not greeted_name.endswith(" ")

Код кажется слишком многословным! Но при наличии тестов можно безбоязненно его порефакторить:

In [ ]:
class Greeter:
    def greet(self, name):
        return "Hello " + name.strip()

In [ ]:
%%ipytest


@pytest.mark.parametrize("name, greeting", [("Mike", "Hello Mike"), ("John", "Hello John"), ("Greg", "Hello Greg")])
def test_greeter(name, greeting):
    assert Greeter().greet(name) == greeting


@pytest.mark.parametrize("name", ["Mike", " Mike", "Mike ", " Mike ", "  Mike", "  Mike  "],
                         ids=["no spaces", "left space", "right space", 
                              "two-side space", "double space", "two-sided double space"])
def test_spaces(name):
    greeted_name = Greeter().greet(name).split(" ", 1)[1]
    assert not greeted_name.startswith(" ") and not greeted_name.endswith(" ")

## 3. Рефакторинг тестов и фикстуры

Сами тесты тоже надо рефакторить. У нас есть две проблемы.

Во-первых, имена тестов не очень информативны. Если упадет тест `test_greater`, будет не совсем понятно, что именно тестировалось и что надо чинить. В целом имена тестам надо давать как можно более подробные — тесты вызываются автоматически, автоматике длина имени безразлична, а вот человеку, читающему выхлоп пайтеста, лучше предоставить как можно больше информации.

[Статья на тему](https://enterprisecraftsmanship.com/posts/you-naming-tests-wrong).


Переименуем наши тесты:

In [ ]:
%%ipytest


@pytest.mark.parametrize("name", ["Mike", "John", "Greg"])
def test_greet_returns_name_with_greeting(name):
    assert Greeter().greet(name) == "Hello " + name


@pytest.mark.parametrize("name", ["Mike", " Mike", "Mike ", " Mike ", "  Mike", "  Mike  "],
                         ids=["no spaces", "left space", "right space", 
                              "two-side space", "double space", "two-sided double space"])
def test_greet_removes_leading_and_trailing_spaces_from_name(name):
    greeted_name = Greeter().greet(name).split(" ", 1)[1]
    assert not greeted_name.startswith(" ") and not greeted_name.endswith(" ")

Вторая проблема — в обоих тестах мы создаем `greeter`. Это приводит к дублированию кода. Кроме того, на практике вместо `greeter` у нас может быть какой-нибудь тяжелый объект типа базы даных, который надо каждый раз инициализировать и чистить. Решить эти проблемы нам поможет механизм фикстур:

In [4]:
%%ipytest

@pytest.fixture(scope="module")
def greeter():
    yield Greeter()

@pytest.mark.parametrize("name", ["Mike", "John", "Greg"])
def test_greet_returns_name_with_greeting(greeter, name):
    assert greeter.greet(name) == "Hello " + name


@pytest.mark.parametrize("name", ["Mike", " Mike", "Mike ", " Mike ", "  Mike", "  Mike  "],
                         ids=["no spaces", "left space", "right space", 
                              "two-side space", "double space", "two-sided double space"])
def test_greet_removes_leading_and_trailing_spaces_from_name(greeter, name):
    greeted_name = greeter.greet(name).split(" ", 1)[1]
    assert not greeted_name.startswith(" ") and not greeted_name.endswith(" ")

UsageError: Cell magic `%%ipytest` not found.


Фикстуры так же могут прибираться за создаваемым объектом в конце теста и иметь разный скоуп, например, создваться на каждый тест, модуль или тред, запускающий тесты. [Подробнее в документации](https://docs.pytest.org/en/stable/fixture.html).

Для БД фикстура может выглядеть примерно так:

In [9]:
%%ipytest -s

class DBConnection:
    pass

class TestDB:
    def init_db(self):
        print("init db")
        
    def get_connection(self):
        return DBConnection()

    def shutdown(self):
        print("close db")


@pytest.fixture(scope="module")
def db_connection():
    db = TestDB()
    db.init_db()
    try:
        yield db.get_connection()
    finally:
        db.shutdown()
    
def test_db_1(db_connection):
    assert db_connection
    
def test_db_2(db_connection):
    assert db_connection

UsageError: Cell magic `%%ipytest` not found.


Так же в `pytest` есть разные встроенные фикстуры. [Список лежит здесь](https://docs.pytest.org/en/stable/fixture.html). Наиболее интересные:
* `monkeypatch` — временно можифицирует методы классов, модулей и т.д.
* `testdir` — создает верменную директорию для каждого теста, которую потом чистит



## 4. Тестирование исключений, сравнение флотов, манкипатчинг

Следующий пункт нашей каты:
- Метод `greet` должен возвращать ошибку если имя - пустая строка (или строка с пробелами)

Для тестирования исключений есть специальная функциональность:

In [ ]:
%%ipytest

@pytest.fixture(scope="module")
def greeter():
    yield Greeter()

def test_greet_raises_value_error_on_empty_string(greeter):
    with pytest.raises(ValueError):
        greeter.greet("")

По тексту отчета видим, что тест ожидал исключения, но его не было. Починим тест:

In [ ]:
class Greeter:
    def greet(self, name):
        name = name.strip()
        if not name:
            raise ValueError("Empty name!")
        return "Hello " + name

In [ ]:
%%ipytest

@pytest.fixture(scope="module")
def greeter():
    yield Greeter()

def test_greet_raises_value_error_on_empty_string(greeter):
    with pytest.raises(ValueError):
        greeter.greet("")

Остался последний пункт нашей каты:
 - Метод `greet` возвращает "Good evening <имя>" если текущее время - 18:00-22:00

Реализация скорее всего будет вызывать `datetime.now()` где-то внутри. Чтобы обеспечить в тесте нужное нам поведение, используем специальную фикстуру `monkeypatch`. Подробно про неё можно почитать [тут](https://docs.pytest.org/en/latest/monkeypatch.html).

In [ ]:
%%ipytest

import datetime

@pytest.fixture(scope="module")
def greeter():
    yield Greeter()

def test_greeting_is_good_evening_in_evening(monkeypatch, greeter):
    fake_time = datetime.datetime.strptime("2007-01-01 19:30:00", "%Y-%m-%d %H:%M:%S")
    class mydatetime:
        @classmethod
        def now(cls):
            return fake_time

    monkeypatch.setattr(datetime, 'datetime', mydatetime)
    assert greeter.greet("Mike").startswith("Good evening")

In [ ]:
import datetime

class Greeter:
    def greet(self, name):
        name = name.strip()
        if not name:
            raise ValueError("Empty name!")
        hour = datetime.datetime.now().hour
        if 18 <= hour <= 22:
            return "Good evening " + name
        return "Hello " + name

In [3]:
%%ipytest

import datetime

@pytest.fixture(scope="module")
def greeter():
    yield Greeter()

def test_greeting_is_good_evening_in_evening(monkeypatch, greeter):
    fake_time = datetime.datetime.strptime("2007-01-01 19:30:00", "%Y-%m-%d %H:%M:%S")
    class mydatetime:
        @classmethod
        def now(cls):
            return fake_time

    monkeypatch.setattr(datetime, 'datetime', mydatetime)
    assert greeter.greet("Mike").startswith("Good evening")

UsageError: Cell magic `%%ipytest` not found.


Посмотрим, что со старыми тестами:

In [ ]:
%%ipytest

@pytest.fixture(scope="module")
def greeter():
    yield Greeter()

@pytest.mark.parametrize("name", ["Mike", "John", "Greg"])
def test_greet_returns_name_with_greeting(greeter, name):
    assert Greeter().greet(name) == "Hello " + name


@pytest.mark.parametrize("name", ["Mike", " Mike", "Mike ", " Mike ", "  Mike", "  Mike  "],
                         ids=["no spaces", "left space", "right space", 
                              "two-side space", "double space", "two-sided double space"])
def test_greet_removes_leading_and_trailing_spaces_from_name(greeter, name):
    greeted_name = greeter.greet(name).split(" ", 1)[1]
    assert not greeted_name.startswith(" ") and not greeted_name.endswith(" ")

Всё падает, потому что cейчас вечер. Выставим для тестов дефолтное время при помощи фикстур:

In [5]:
%%ipytest

import datetime


@pytest.fixture(scope="function")
def set_time(monkeypatch):
    def set_time_(time):
        class mydatetime:
            @classmethod
            def now(cls):
                return time

        monkeypatch.setattr(datetime, 'datetime', mydatetime)
    yield set_time_

    
@pytest.fixture(scope="function")
def set_day_time(set_time):
    yield set_time(datetime.datetime.strptime("2007-01-01 10:30:00", "%Y-%m-%d %H:%M:%S"))
    

@pytest.mark.parametrize("name", ["Mike", "John", "Greg"])
@pytest.mark.usefixtures("set_day_time")
def test_greet_returns_name_with_greeting(greeter, name):
    assert greeter.greet(name) == "Hello " + name


@pytest.mark.parametrize("name", ["Mike", " Mike", "Mike ", " Mike ", "  Mike", "  Mike  "],
                         ids=["no spaces", "left space", "right space", 
                              "two-side space", "double space", "two-sided double space"])
@pytest.mark.usefixtures("set_day_time")
def test_greet_removes_leading_and_trailing_spaces_from_name(greeter, name):
    greeted_name = greeter.greet(name).split(" ", 1)[1]
    assert not greeted_name.startswith(" ") and not greeted_name.endswith(" ")
    

def test_greeting_is_good_evening_in_evening(set_time, monkeypatch, greeter):
    set_time(datetime.datetime.strptime("2007-01-01 19:30:00", "%Y-%m-%d %H:%M:%S"))
    assert greeter.greet("Mike").startswith("Good evening")

UsageError: Cell magic `%%ipytest` not found.


#### Сравнение float

 Сравнение `float` сталось за кадром, разберем его отдельно.
 Из-за ошибок округления `float` трудно сравнивать через `==`.

In [ ]:
%%ipytest
def test_float():
    assert 0.1 + 0.2 == 0.3

Исправить ситуацию поможет `pytest.approx`:

In [ ]:
%%ipytest
def test_float():
    assert 0.1 + 0.2 == pytest.approx(0.3)

`pytest.approx` работает и с коллекциями:

In [ ]:
%%ipytest
def test_float():
    assert [0.1 + 0.2, 0.5] == pytest.approx([0.3, 0.5])

#### Манкипатчинг модуля requests

In [ ]:
import requests


def get_json(url):
    """Takes a URL, and returns the JSON."""
    r = requests.get(url)
    return r.json()

Внутри теста очень не хочется ходить по сети, при этом тест написать надо. Воспользуемся `monkeypatch`:

In [ ]:
%%ipytest -s

# импортим модуль requests чтобы потом его модифицировать
import requests

# делаем mock на  response-объект библиотеки requests
class MockResponse:
    @staticmethod
    def json():
        return {"mock_key": "mock_response"}


def test_get_json(monkeypatch):

    # Делаем фальшивый метод get
    def mock_get(*args, **kwargs):
        return MockResponse()

    # Подменяем настоящий get на фальшивый
    monkeypatch.setattr(requests, "get", mock_get)

    # Тестируем наш метод
    result = get_json("https://fakeurl")
    assert result["mock_key"] == "mock_response"

Для манкипатчинга модуля requests есть отдельная библиотека — [requests-mock](https://requests-mock.readthedocs.io/en/latest/).

## Итого

Мы сделали небольшую кату, познакомились с TDD и основной функциональностью `pytest`:

* Как pyteset находит тесты
* Ассерты pyteset
* Параметризация тестов
* Фикстуры 
* Тестирование исключений
* Манкипатчинг

Какие в итоге профиты у тестов:
 - Тесты помогают следить за тем, что код соответствует спецификации
 - Тесты позволяют рефакторить код и не бояться при этом посадить баг
 - Тесты документируют код
 
Что осталось за кадром:
 - Виды тестов — юнит, интеграционные и т.д.
 - Настройка тестов в ci/cd
 - Плагины pytest